## Versión 3.0
En esta nueva versión intentaré encontrar una solución al problema de los muchos códigos con dummy.  
Lo que ahora voy a hacer es crear un diccionario con los campos indicados.
El escritor lo que va a hacer es recibir el diccionario de cada venta y él se encargará de transformarlo en texto.

Como ya he aprendido suficiente voy a resumir algo. En este apartado empezaré a simplificarlo todo. Meteré lo que tengo hecho en un fichero y comenzaré a juntar código.  

Así debe quedar todo más limpio.

In [1]:
import os
import pandas as pd
# Pandas es la librerá que me premite trabajar con datasets ... creo.


In [2]:
class Venta():
    """Es la instancia que usaré para el Wrapper (¿?)"""
    i = 1 
    #atributosGenerales = dict() # atribudo de instancia
    #lineasBrutas = list()
    #lineaBrutaCabecera = ''
    #lineasBrutasLineasVentas = []
    #lineasDeVenta = list()
    #codigosVendidos = dict()
    
    def __init__(self):
        self.atributosGenerales = dict() # atribudo de instancia
        self.lineasBrutas = list()
        self.lineaBrutaCabecera = ''
        self.lineasBrutasLineasVentas = []
        self.lineasDeVenta = list()
        self.codigosVendidos = dict()
        self.i = self.i + 1
        
    def __str__(self):
        # No entiendo la diferencia con __repr__() que veo en 
        #              https://docs.python.org/3/library/functions.html#repr
        # Aquí lo que quiero es llamar a cada variable y nombrarla con su atributo
        # si puedo hacer eso ya tengo controlado el acceso a todas las variables 
        # aun sin conocer su nombre.
        #for key,value in dict(self):
        #    texto = texto + '' 
        # en https://docs.python.org/3/library/functions.html#repr  hay documentación iinteresante
        # como es 
        texto = '--------->' + self.__class__.__name__ + '<-------------- ' + str(self.i) +'\n'
        texto = texto + self.lineaBrutaCabecera + '\n'
        for linea in self.lineasDeVenta:
            print(linea)
        texto =  texto + '\n fecha: ' + self.fecha    
        texto = texto + '\n hora: '+ self.hora     
        texto = texto + '\n vendedor: '+ self.vendedor     
        texto = texto + '\n codigoCliente: '+ self.codigoCliente    
        texto = texto + '\n bruto: '+ self.bruto     
        texto = texto + '\n descuento: '+ self.descuento    
        texto = texto + '\n neto: '+ self.neto    
        texto = texto + '\n codigoVenta: '+ self.codigoVenta    
        texto = texto + '\n numFactura: '+ self.numFactura    
        texto = texto + '\n aCuenta: '+ self.aCuenta    
        texto = texto + '\n entrega: '+ self.entrega    
        texto = texto + '\n devolucion: '+ self.devolucion
        return texto 
    
    def setBrutas(self, lineaTexto = ''):
        self.lineasBrutas = lineaTexto
        return self
    
    def asignarBrutas(self, lineaTexto = ''):
        self.lineasBrutas = lineaTexto
        self.asignoCabecera()
        self.asignoLineasVenta()
        return self
    
    def asignoCabecera(self): # método de instancia
        # selecciono la línea de cabecera
        # extraigo de cada separación de tabulador la información necesaria.
        self.lineaBrutaCabecera = self.getLineaBrutaCabecera()
        tokens = self.lineaBrutaCabecera.split('\t')
        
        atributos = ['Fecha','Hora','Vendedor','Codigo','ClienteDescripción','Tipo','TA','Uni.','P.Ant.','P.V.P.','Imp. Bruto','Dto.', 'Imp. Neto','Numero Doc.','R.P.','Fact.','A Cuenta','Entrega','Devoluc.','Tipo de Pago']
        relacion = dict(zip(atributos,tokens))
        self.fecha = relacion[ 'Fecha'] # '01/07/2019'
        self.hora = relacion[ 'Hora'] #: '9:05'
        self.vendedor = relacion[ 'Vendedor'] #: '(20)2-ARACELI'
        self.codigoCliente = relacion[ 'Codigo' ]  #: ''
        self.bruto = relacion[ 'Imp. Bruto' ]  #: '17,14 €'
        self.descuento = relacion['Dto.' ]  #: '0,00 €'
        self.neto = relacion['Imp. Neto' ]  #: '17,14 €'
        self.codigoVenta = relacion[ 'Numero Doc.' ]  #: 'A155690/2019'
        self.numFactura = relacion[ 'Fact.' ]  #: ''
        self.aCuenta = relacion['A Cuenta' ] #: '0,00 €', 
        self.entrega = relacion['Entrega' ]  # '20,00 €'
        self.devolucion = relacion['Devoluc.' ]  # '2,86 €'
        return self
    
    def getLineaBrutaCabecera(self):
        #self.lineasBrutas select: [:alinea | alinea[0] != '\t']
        seleccionar = lambda aLineaTxt : aLineaTxt[0] != '\t'
        cabecera = list(filter( seleccionar , self.lineasBrutas ))
        return cabecera[0]
    
    def asignoLineasVenta(self):
        self.lineasBrutasLineasVentas = self.getLineasBrutasLineasVentas()
        lineasDeLaVenta = []
        for linea in self.lineasBrutasLineasVentas :
            lineaFormateada = LineaVenta()
            lineaFormateada.absorbeBruta(linea)
            if lineaFormateada.codigoNacional in self.codigosVendidos : # para saber los CN y sus cantidades
                self.codigosVendidos[lineaFormateada.codigoNacional] = self.codigosVendidos[lineaFormateada.codigoNacional] + lineaFormateada.unidades
            else:
                self.codigosVendidos[lineaFormateada.codigoNacional] = lineaFormateada.unidades
            lineasDeLaVenta.append(lineaFormateada)
        self.lineasDeVenta = lineasDeLaVenta
        return lineasDeLaVenta
    
    def getLineasBrutasLineasVentas(self):
        #¿Cómo se hace para eliminar de una listas un objeto.
        # o mejor dícho para eliminar de una lista una serie de objetos.
        listaTempo = self.lineasBrutas.copy()
        listaTempo.remove( self.lineaBrutaCabecera )
        self.lineasBrutasLineasVentas = listaTempo
        return listaTempo
        


In [3]:
class LineaVenta():
    """Es la instancia que usaré para el Wrapper (¿?)"""
    lineaTextoBruta = ''
    def __str__(self):
        texto = '--------->' + self.__class__.__name__ + '<--------------' + '\n'
        texto = texto + 'C.N. :  ' + self.codigoNacional + ' --- '
        texto = texto + 'Und. :  ' + self.unidades + ' --- '
        texto = texto + 'descripcion :  ' + self.descripcion + '\t'
        texto = texto + 'tipoAportacion :  ' + self.tipoAportacion + '\t'
        texto = texto + 'precioAnterior :  ' + self.precioAnterior + '\t'
        texto = texto + 'bruto :  ' + self.bruto + '\t' 
        texto = texto + 'descuento :  ' + self.descuento + '\t' 
        texto = texto + 'neto :  ' + self.neto + '\t'  
        texto = texto + 'codigoVenta :  ' + self.codigoVenta + '\t'
        return texto

    def setBruta(self, lineaTexto):
        self.lineaTextoBruta = lineaTexto
    
    def absorbeBruta(self, lineaTexto):
        self.setBruta(lineaTexto)
        self.asignoDelTexto()
        return self 
    
    def asignoDelTexto(self): # método de instancia
        # selecciono la línea de cabecera
        # extraigo de cada separación de tabulador la información necesaria.
        tokens = self.lineaTextoBruta.split('\t')
        
        atributos = ['Fecha','Hora','Vendedor','Codigo','ClienteDescripción','Tipo','TA','Uni.','P.Ant.','P.V.P.','Imp. Bruto','Dto.', 'Imp. Neto','Numero Doc.','R.P.','Fact.','A Cuenta','Entrega','Devoluc.','Tipo de Pago']
        relacion = dict(zip(atributos,tokens))
         
        self.codigoNacional = relacion[ 'Codigo' ] 
        self.descripcion = relacion[ 'ClienteDescripción' ]
        self.unidades = relacion[ 'Uni.' ]
        self.tipoAportacion = relacion[ 'TA' ]
        self.precioAnterior = relacion[ 'P.Ant.' ]
        self.bruto = relacion[ 'Imp. Bruto' ]
        self.descuento =  relacion[ 'Dto.' ] 
        self.neto =  relacion[ 'Imp. Neto' ] 
        self.codigoVenta =  relacion[ 'Numero Doc.' ]
        return self



In [4]:
"Ahora a pasar el objeto Venta a texto. Tengo tres clases. Dos heredan de ToCSV"
"Ha sido todo un aprendizaje el hacer todo esto. Ahora tengo un montón de anotaciones en mis apuntes"
"La clase a repasar para ver lo aprendido es VentaToCSVDummy"
class ToCSV():
    """Es la instancia que usaré para el Wrapper (¿?)"""
    
    texto = ''
    def __str__(self):
        texto = '--------->' + self.__class__.__name__ + '<---------' 
        return texto
    
    def pasarALineaConCabecera(self):
        self.pasarATextoCabecera()
        self.pasarALinea()
        return self.texto
    
    def pasarALinea(self):
        self.pasarATextoVenta()
        self.pasarATextoLineas()
        return self.texto


class VentaToCSV(ToCSV):
    """Es la instancia que usaré para el Wrapper (¿?)"""
    venta = ''
    
    def venta(self, venta):
        self.venta = venta
        return self
        
    def pasarATextoCabecera(self):
        txt = ''
        txt = txt + 'fecha' + '\t'
        txt = txt + 'hora'  + '\t'
        txt = txt + 'vendedor'  + '\t'
        txt = txt + 'codigoCliente' + '\t'
        txt = txt + 'bruto' + '\t'
        txt = txt + 'descuento' + '\t'
        txt = txt + 'neto'  + '\t'
        txt = txt + 'codigoVenta'  + '\t'
        txt = txt + 'numFactura' + '\t'
        txt = txt + 'aCuenta' + '\t'
        txt = txt + 'entrega'  + '\t'
        txt = txt + 'devolucion' + '\t'
        self.texto = self.texto + txt + '\n'  
        return txt 
        
    def pasarATextoVenta(self):
        txt = ''
        txt = txt + self.venta.fecha + '\t'
        txt = txt + self.venta.hora  + '\t'
        txt = txt + self.venta.vendedor  + '\t'
        txt = txt + self.venta.codigoCliente + '\t'
        txt = txt + self.venta.bruto + '\t'
        txt = txt + self.venta.descuento + '\t'
        txt = txt + self.venta.neto  + '\t'
        txt = txt + self.venta.codigoVenta  + '\t'
        txt = txt + self.venta.numFactura + '\t'
        txt = txt + self.venta.aCuenta + '\t'
        txt = txt + self.venta.entrega  + '\t'
        txt = txt + self.venta.devolucion + '\t'
        self.texto = self.texto + txt  
        return txt
    
    def pasarATextoLineas(self):
        txt = ''
        codigos = set()
        for linea in self.venta.lineasDeVenta :
            codigos.add( linea.codigoNacional)
        for codigo in codigos :
            txt = txt + codigo  +  '\t'
        self.texto = self.texto + txt  
        return txt


class VentaToCSVDummy(VentaToCSV):
    """Es la instancia que usaré para el Wrapper (¿?)"""
    ordenDeAtributos =[ 'hora', 'vendedor', 'codigoCliente', 'bruto', 'descuento', 'neto', 'codigoVenta', 'numFactura', 'aCuenta', 'entrega', 'devolucion', '687016', '890533', '888891']
    def aTextoSinCabecera(self, ordenDeAtributos =  [ 'hora', 'vendedor', 'codigoCliente', 'bruto', 'descuento', 'neto', 'codigoVenta', 'numFactura', 'aCuenta', 'entrega', 'devolucion', '687016', '890533', '888891']):
        txt = ''
        for atributo in ordenDeAtributos:
            if atributo in self.venta.__dict__.keys(): # dict__me da un listado de los nombres de variables de venta
                txt = txt + getattr( self.venta,   atributo) + '\t' # getattr da el valor de atributo en obj venta
            else:
                txt = txt +  'NA' + '\t'
        txt = txt + '\n'
        self.texto = self.texto + txt  
        return txt
    
    def aTextoConCabecera(self, ordenDeAtributos =  [ 'hora', 'vendedor', 'codigoCliente', 'bruto', 'descuento', 'neto', 'codigoVenta', 'numFactura', 'aCuenta', 'entrega', 'devolucion', '687016', '890533', '888891']):
        txt = ''
        txt = txt + self.textoDeCabecera(ordenDeAtributos = ordenDeAtributos) + '\n'
        txt = txt + self.aTextoSinCabecera(ordenDeAtributos = ordenDeAtributos)
        self.texto = self.texto + txt  
        return txt 
    
    def textoDeCabecera(self, ordenDeAtributos ):
        txt = ''
        for cabecera in ordenDeAtributos :
            txt = txt + cabecera + '\t' 
        return txt 
    
    def textoAtributos(self, ordenDeAtributos =  [ 'hora', 'vendedor', 'codigoCliente', 'bruto', 'descuento', 'neto', 'codigoVenta', 'numFactura', 'aCuenta', 'entrega', 'devolucion', '687016', '890533', '888891']):
        txt = ''
        for atributo in ordenDeAtributos:
            if atributo in self.venta.__dict__.keys(): # dict__me da un listado de los nombres de variables de venta
                txt = txt + getattr( self.venta,   atributo) + '\t' # getattr da el valor de atributo en obj venta
            else:
                txt = txt +  'NA' + '\t'  
        return txt
    
    def textoCodigos(self, ordenDeCodigos =  [  '687016', '890533']):
        txt = ''
        for codigoTxt in ordenDeCodigos:
            if codigoTxt in self.venta.codigosVendidos :
                txt = txt + self.venta.codigosVendidos[codigoTxt] + '\t' # getattr da el valor de atributo en obj venta
            else:
                txt = txt +  'NA' + '\t'  
        return txt
    def aTextoSinCabecera2(self, ordenDeAtributos =  [ 'hora', 'vendedor', 'codigoCliente', 'bruto', 'descuento', 'neto', 'codigoVenta', 'numFactura', 'aCuenta', 'entrega', 'devolucion', '687016', '890533', '888891']):
        soloAtributos = [valor for valor in ordenDeAtributos if not valor.isnumeric() ]
        soloCodigos = [valor for valor in ordenDeAtributos if valor not in soloAtributos ]
        txt = self.textoAtributos(soloAtributos)
        txt = txt + self.textoCodigos(soloCodigos)
        self.texto = self.texto + txt  
        return txt



In [14]:
class MiMain():
    """ es la clase principal."""
    
    pathLocal = '/Users/luissanchez/AnacondaProjects/cursoAIJuanGabrielGomila/'
    pathRelativo = 'python-ml-course/luis/'
    #nombreFichero = 'ventas1.csv'
    nombreFichero = 'ventasprueba.csv'
    #nombreFichero = 'ventaspruebacoma.csv'
    #nombreFichero = 'ventaspruebacomatab.csv'
    pathCompleto = pathLocal + pathRelativo + nombreFichero
    
    def __init__(self):
        ventas = self.leoFichero()
        self.saveVentas(ventas) 
    
    def leoFichero(self):
        """Lee el fichero y luego devuelve tantas ventas (instancias de Venta) como haya encontrado."""
        # Abro el fichero
        fichero = open( self.pathCompleto ,'r')

        # de la docu oficial: https://docs.python.org/3/tutorial/inputoutput.html
        ficheroInverso = (fichero.readlines())[::-1]

        # Ahora voy cargando líneas hasta que encuentre un resumen de venta.
        ventas = list()
        buffer = list()
        for linea in ficheroInverso:
            buffer.append(linea)  # todas las líneas se acumulan como válidas
            if linea[0] != '\t':  # pero solo las que tienen el primer campo vacío cierran la venta.
                venta = Venta()
                venta.asignarBrutas(buffer)
                ventas.append(venta) # añado la venta recien extraida.
                buffer = list() # para dejar vacío para la próxima venta.
        fichero.close()
        return ventas
    
    def saveVentas(self, ventas):
        """PAsa a fichero las ventas"""
        #ordenColumnas = [ 'hora', 'vendedor', 'codigoCliente', 'bruto', 'descuento', 'neto', 'codigoVenta', 'numFactura', 'aCuenta', 'entrega', 'devolucion', '687016', '890533', '888891']
        #ordenColumnas = [ 'hora', 'vendedor', 'bruto', 'neto', 'codigoVenta', 'entrega', 'devolucion']
        ordenColumnas = [ 'hora','neto', 'codigoVenta', '687016', '890533', '888891']
        print('\t'.join(ordenColumnas))
        for venta in ventas:
            print(VentaToCSVDummy().venta(venta).aTextoSinCabecera2(ordenDeAtributos = ordenColumnas ))
            
        return self
    


In [15]:
MiMain()

# pd.merge( dsUno, dsDos, how='outer' , on='codigoVenta' ) 
# on es la columna índice. 
# how='outer' es para que sume filas y columnas


hora	neto	codigoVenta	687016	890533	888891
11:10	16,18 €	A155713/2019	11	NA	NA	
10:49	8,95 €	A155712/2019	NA	NA	NA	
10:48	3,47 €	A155711/2019	NA	1	1	
10:24	16,60 €	A155706/2019	NA	NA	NA	
10:07	15,05 €	A155701/2019	NA	NA	NA	
9:44	0,66 €	A155696/2019	NA	NA	NA	
9:44	3,12 €	A155695/2019	NA	NA	NA	
9:31	11,02 €	A155694/2019	NA	NA	NA	
9:25	9,71 €	A155693/2019	NA	NA	NA	
9:24	62,96 €	A155692/2019	NA	NA	NA	
9:06	9,71 €	A155691/2019	NA	NA	NA	
9:05	17,14 €	A155690/2019	NA	NA	NA	
Hora	Imp. Neto	Numero Doc.	NA	NA	NA	


In [13]:
[1, 2])

TypeError: sequence item 0: expected str instance, int found